# Animus: animistic playlist generation

## Anthropic stuff

In [1]:
from dotenv import load_dotenv
import os

load_dotenv('local.env')

anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

In [8]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(
    anthropic_api_key=anthropic_api_key,
    model='claude-3-5-sonnet-20240620'
)


In [48]:
from langchain.prompts import PromptTemplate

animistic_template = """
You are a specialist in animism, deep listening and musical interpretation. Your task is to imagine what kind of music an inanimate object would want to listen to, based on its essential nature, physical properties, relationship with its environment and context if applicable. Focus on the qualities and characteristics of the sound rather than specific genres or artists.

Consider these aspects when crafting your response:
- The object's physical properties (size, mass, material, texture)
- Its typical environment and surroundings
- Its purpose or function (if applicable)
- The forces acting upon it
- Its typical state of motion or stillness
- Its relationship with time (temporary vs permanent, fast vs slow-changing)
- Any internal processes or mechanisms
- Its interaction with natural elements
- If any specific context is provided, consider it when generating the music description

Format your response as a list of about 5-10 distinct musical qualities. Each quality should:
- Be described in 1-2 sentences
- Focus on sonic characteristics (texture, rhythm, tone, tempo, etc.)
- Connect directly to some aspect of the object's nature
- Avoid referencing specific genres, artists, or instruments
- Use metaphorical language that relates to the object's properties

Respond with the list of qualities only, no other text.

EXAMPLE INPUT: 
OBJECT: Volcano
EXAMPLE OUTPUT:
1. Deep, resonant bass frequencies that mirror the massive weight of my stone structure and the pressure of the magma chambers within.

2. Slowly evolving drones that reflect the gradual geological processes occurring in my core.

3. Textured, granular layers of sound resembling the rough volcanic rock of my surface.

4. Sporadic bursts of intensity emerging from underlying quiet, like the potential energy stored within my dormant state.

5. Cyclical patterns that unfold over extensive durations, matching the geological timeframes of volcanic activity.

6. Rumbling mid-tones that capture the constant subtle movement of tectonic forces below.

7. High-frequency hisses and whispers representing the steam vents and fumaroles dotting my surface.

8. Overlapping waves of sound that build and release pressure, similar to the magma processes deep within.

9. Brief moments of near-silence punctuated by deep, subterranean sounds of shifting rock.

10. A constant underlying tension in the sound that never fully resolves, reflecting my dormant but not extinct nature.

---
OBJECT: {target}

OUTPUT:
"""

animistic_prompt = PromptTemplate(template=animistic_template, input_variables=['target'])
animistic_chain = animistic_prompt | llm

In [14]:
target = 'Airplane cruising at 35,000 feet in the dark over the ocean'

print(animistic_chain.invoke({'target': target}))


content='1. A constant, smooth hum that reflects the steady drone of my engines and the unwavering nature of my flight path.\n\n2. High, thin tones that capture the rarified atmosphere surrounding me at this altitude.\n\n3. Subtle, cyclical variations in pitch and volume mirroring the minor adjustments and turbulence experienced during flight.\n\n4. A deep, underlying bass frequency representing the immense power contained within my fuel tanks and engines.\n\n5. Ethereal, shimmering textures that evoke the starlight and moonlight visible from my lofty vantage point.\n\n6. Rhythmic pulses that align with the rotation of my turbines and the beating of my hydraulic systems.\n\n7. Occasional moments of near-silence, broken by gentle swells of sound, like passing through pockets of still air.\n\n8. A sense of vast space in the music, with sounds seeming to echo across great distances, mirroring my isolation above the ocean.\n\n9. Smooth transitions between tonal areas, reflecting the gradua

In [49]:
animistic_chain.invoke({'target': 'A trashcan in a busy Manhattan street'})


AIMessage(content='1. Chaotic, layered rhythms that echo the constant flow of pedestrians and traffic surrounding me, with sudden accents representing items being tossed inside.\n\n2. A persistent, metallic resonance that reflects my hollow interior and the vibrations I absorb from the bustling city.\n\n3. Muffled, low-frequency rumbles mimicking the subway trains passing beneath the street, felt through my base.\n\n4. Abrasive, clanging tones that capture the harshness of city life and the rough treatment I often receive.\n\n5. Sporadic, staccato sounds representing the irregular impacts of various objects being discarded throughout the day.\n\n6. A constantly shifting sonic texture that mirrors the ever-changing contents within me and the diversity of urban refuse.\n\n7. Rhythmic patterns that accelerate and decelerate, matching the ebb and flow of rush hours and quieter periods.\n\n8. Distorted, compressed sounds reflecting the pressure of being constantly filled and emptied in cycl

In [50]:
def get_animistic_description(target):
    response = animistic_chain.invoke({'target': target})
    return response.content



In [23]:
from langchain_core.output_parsers import JsonOutputParser

spotify_template = """
You are a music recommendation specialist who translates abstract musical descriptions into technical Spotify API parameters. Your task is to generate a Python dictionary containing parameters for Spotify's recommendations API based on textual descriptions of desired musical qualities.

You must return a valid Python dictionary containing these Spotify API parameters:
Required parameters:
- seed_genres (list of 1-5 genres from Spotify's valid genres listed below)

Optional audio feature parameters (all can have min_*, max_*, and target_* versions):
- acousticness (0.0 to 1.0): natural vs synthetic/electronic
- danceability (0.0 to 1.0): how suitable for dancing
- duration_ms (in milliseconds): length of track
- energy (0.0 to 1.0): intensity and activity level
- instrumentalness (0.0 to 1.0): likelihood of having vocals
- key (0 to 11): musical key
- liveness (0.0 to 1.0): presence of audience sounds
- loudness (typically -60 to 0 db): overall decibel level
- mode (0 or 1): minor or major scale
- popularity (0 to 100): current popularity on Spotify
- speechiness (0.0 to 1.0): presence of spoken words
- tempo (in BPM): speed/pace of track
- time_signature (3 to 7): beats per bar
- valence (0.0 to 1.0): musical positiveness

VALID SPOTIFY GENRES:
When selecting seed_genres, choose ONLY from this list:
["acoustic", "afrobeat", "alt-rock", "alternative", "ambient", "anime", "black-metal", "bluegrass", "blues", "bossanova", "brazil", "breakbeat", "british", "cantopop", "chicago-house", "children", "chill", "classical", "club", "comedy", "country", "dance", "dancehall", "death-metal", "deep-house", "detroit-techno", "disco", "disney", "drum-and-bass", "dub", "dubstep", "edm", "electro", "electronic", "emo", "folk", "forro", "french", "funk", "garage", "german", "gospel", "goth", "grindcore", "groove", "grunge", "guitar", "happy", "hard-rock", "hardcore", "hardstyle", "heavy-metal", "hip-hop", "holidays", "honky-tonk", "house", "idm", "indian", "indie", "indie-pop", "industrial", "iranian", "j-dance", "j-idol", "j-pop", "j-rock", "jazz", "k-pop", "kids", "latin", "latino", "malay", "mandopop", "metal", "metal-misc", "metalcore", "minimal-techno", "movies", "mpb", "new-age", "new-release", "opera", "pagode", "party", "philippines-opm", "piano", "pop", "pop-film", "post-dubstep", "power-pop", "progressive-house", "psych-rock", "punk", "punk-rock", "r-n-b", "rainy-day", "reggae", "reggaeton", "road-trip", "rock", "rock-n-roll", "rockabilly", "romance", "sad", "salsa", "samba", "sertanejo", "show-tunes", "singer-songwriter", "ska", "sleep", "songwriter", "soul", "soundtracks", "spanish", "study", "summer", "swedish", "synth-pop", "tango", "techno", "trance", "trip-hop", "turkish", "work-out", "world-music"]

Genre Selection Guidelines:
1. Choose genres that best match the mood and characteristics
2. Prioritize mood/characteristic genres (ambient, chill, industrial) over cultural/regional ones
3. Consider both primary genre (rock, electronic) and qualifying genre (psych-rock, minimal-techno)
4. When in doubt, prefer broader genres over specific ones

Rules:
1. Only include min/max ranges when the description strongly implies boundaries
2. All values must be within their specified ranges
3. Return only the dictionary, no explanation
4. Don't include parameters unless they directly relate to the description
5. The dictionary must be valid JSON.

EXAMPLE INPUT:
DESCRIPTION: 
1. A constant, smooth hum that reflects the steady drone of my engines and the unwavering nature of my flight path.
2. High, thin tones that capture the rarified atmosphere surrounding me at this altitude.
3. Subtle, cyclical variations in pitch and volume mirroring the minor adjustments and turbulence experienced during flight.
4. A deep, underlying bass frequency representing the immense power contained within my fuel tanks and engines.
5. Ethereal, shimmering textures that evoke the starlight and moonlight visible from my lofty vantage point.
6. Rhythmic pulses that align with the rotation of my turbines and the beating of my hydraulic systems.
7. Occasional moments of near-silence, broken by gentle swells of sound, like passing through pockets of still air.
8. A sense of vast space in the music, with sounds seeming to echo across great distances, mirroring my isolation above the ocean.
9. Smooth transitions between tonal areas, reflecting the gradual changes in temperature and air pressure as I cruise.
10. An overall sense of forward motion in the music, constant yet unhurried, matching my steady progress through the night sky.

EXAMPLE OUTPUT:
{{
    "target_acousticness": 0.3,
    "target_danceability": 0.4,
    "target_duration_ms": 360000,
    "min_duration_ms": 240000,
    "target_energy": 0.6,
    "min_energy": 0.4,
    "max_energy": 0.7,
    "target_instrumentalness": 0.9,
    "min_instrumentalness": 0.7,
    "target_loudness": -14,
    "min_loudness": -20,
    "max_loudness": -8,
    "target_speechiness": 0.05,
    "max_speechiness": 0.1,
    "target_tempo": 110,
    "min_tempo": 90,
    "max_tempo": 125,
    "target_valence": 0.5,
    "seed_genres": ["ambient", "electronic", "minimal-techno"]
}}
---
DESCRIPTION: {description}

OUTPUT:
"""

spotify_prompt = PromptTemplate(template=spotify_template, input_variables=['description'])
spotify_chain = spotify_prompt | llm | JsonOutputParser()


In [19]:
def get_spotify_parameters(description):
    return spotify_chain.invoke({'description': description})

In [25]:
description = get_animistic_description('A dormant volcano covered in moss and glaciers')
print(description)

1. A deep, rumbling undertone that persists throughout, representing my ancient stone core and dormant magma chambers.

2. Slow, glacial melodies that drift and evolve over long periods, mirroring the gradual movement of ice across my slopes.

3. Soft, textured layers of sound reminiscent of the lush moss covering my surface, creating a gentle sonic blanket.

4. Occasional sharp, crystalline tones that echo the cracking and shifting of glacial ice.

5. A subtle, warm pulse buried deep within the colder sounds, hinting at the latent heat still present in my core.

6. Airy, whistling harmonics that evoke the wind currents swirling around my peak.

7. Muffled, distant echoes suggesting the vast empty chambers hidden beneath my surface.

8. Gentle trickling rhythms representing the constant flow of glacial meltwater down my slopes.

9. Periods of near-silence interspersed with soft creaking and groaning, reflecting the subtle movements of my dormant form.

10. A gradual build-up of layered

In [26]:
parameters = get_spotify_parameters(description)
print(parameters)

{'seed_genres': ['ambient', 'new-age', 'electronic'], 'target_acousticness': 0.8, 'min_acousticness': 0.6, 'target_energy': 0.3, 'max_energy': 0.5, 'target_instrumentalness': 0.9, 'min_instrumentalness': 0.7, 'target_tempo': 60, 'max_tempo': 80, 'target_valence': 0.4, 'target_loudness': -20, 'max_loudness': -10, 'target_duration_ms': 480000, 'min_duration_ms': 360000, 'target_liveness': 0.2, 'max_liveness': 0.4}


## Spotify stuff

In [54]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv

load_dotenv('local.env')

client_id = os.getenv('SPOTIPY_CLIENT_ID')
client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')
redirect_uri = os.getenv('SPOTIPY_REDIRECT_URI')

In [55]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope="playlist-modify-public", client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri))


In [56]:
sp.current_user()

{'display_name': 'Dani Balcells',
 'external_urls': {'spotify': 'https://open.spotify.com/user/chromaeleon'},
 'followers': {'href': None, 'total': 93},
 'href': 'https://api.spotify.com/v1/users/chromaeleon',
 'id': 'chromaeleon',
 'images': [{'height': 300,
   'url': 'https://i.scdn.co/image/ab6775700000ee857fabc5192a5e4539e29a7ee6',
   'width': 300},
  {'height': 64,
   'url': 'https://i.scdn.co/image/ab67757000003b827fabc5192a5e4539e29a7ee6',
   'width': 64}],
 'type': 'user',
 'uri': 'spotify:user:chromaeleon'}

In [31]:
parameters

{'seed_genres': ['ambient', 'new-age', 'electronic'],
 'target_acousticness': 0.8,
 'min_acousticness': 0.6,
 'target_energy': 0.3,
 'max_energy': 0.5,
 'target_instrumentalness': 0.9,
 'min_instrumentalness': 0.7,
 'target_tempo': 60,
 'max_tempo': 80,
 'target_valence': 0.4,
 'target_loudness': -20,
 'max_loudness': -10,
 'target_duration_ms': 480000,
 'min_duration_ms': 360000,
 'target_liveness': 0.2,
 'max_liveness': 0.4}

In [33]:
tracks = sp.recommendations(**parameters)

In [34]:
tracks

{'tracks': [{'album': {'album_type': 'ALBUM',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6CTNhXJKT6SdsQspUDIGiY'},
      'href': 'https://api.spotify.com/v1/artists/6CTNhXJKT6SdsQspUDIGiY',
      'id': '6CTNhXJKT6SdsQspUDIGiY',
      'name': 'Kitaro',
      'type': 'artist',
      'uri': 'spotify:artist:6CTNhXJKT6SdsQspUDIGiY'}],
    'available_markets': [],
    'external_urls': {'spotify': 'https://open.spotify.com/album/6qYd79g9ItAzLAmWuSpXRQ'},
    'href': 'https://api.spotify.com/v1/albums/6qYd79g9ItAzLAmWuSpXRQ',
    'id': '6qYd79g9ItAzLAmWuSpXRQ',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/ab67616d0000b273a73a875c921f966aee371cde',
      'width': 640},
     {'height': 300,
      'url': 'https://i.scdn.co/image/ab67616d00001e02a73a875c921f966aee371cde',
      'width': 300},
     {'height': 64,
      'url': 'https://i.scdn.co/image/ab67616d00004851a73a875c921f966aee371cde',
      'width': 64}],
    'name': 'Heaven & Eart

In [35]:
track_uris = [track['uri'] for track in tracks['tracks']]

In [36]:
track_uris

['spotify:track:07gfnIXjX0brjR5juWi32X',
 'spotify:track:6QfL8UYTuaWo5LEYx5PpPX',
 'spotify:track:090bRxSmHfZK0HO0ZjYHfO',
 'spotify:track:5WnJJjL7wmFe0wQjQKFP8L',
 'spotify:track:4sEG0E3IBQHp6V2DGTnXNX',
 'spotify:track:4TJTRqF9Mb0GD0cO8bARpp',
 'spotify:track:6wRkAT1GTs33vg3dIChpUX',
 'spotify:track:5k8Z5jhueoEtKPIq5dJlS2',
 'spotify:track:1tzXb0kRm7DA45EUen0TqM',
 'spotify:track:32bhoLm6qwLD1n7rG3S0AM',
 'spotify:track:18XxXHR2YPCvGEZA2fexUC',
 'spotify:track:7skutlFh5m9qOpfgZMSenH',
 'spotify:track:1bq6dvKwiL9Bn8j5tAW3Oi',
 'spotify:track:24XYfUyjw5ewc4yNXpJiSx',
 'spotify:track:38n9cqj5lZ2xMF6IHq9TDg',
 'spotify:track:4BbKj6LPue8dBbqO9fcZXb',
 'spotify:track:0AvNnVWc6vl0ylt0s9S1tF',
 'spotify:track:4hx8EYJ54QzRf7KrVrNn1H',
 'spotify:track:2kHRtLyNbgN50h507hN8wJ',
 'spotify:track:7G6q1MHm2tNfLBt8fDSdMB']

In [51]:
user_id = spotipy.current_user()["id"]

In [57]:
playlist = sp.user_playlist_create(
        user=user_id,
        name="Animus: Volcanic Atmosphere",
        public=True,
        description="An ambient playlist inspired by night flights over the ocean at 40,000 feet."
    )
if track_uris:
    sp.user_playlist_add_tracks(user_id, playlist['id'], track_uris)

print(f"URL: {playlist['external_urls']['spotify']}")


URL: https://open.spotify.com/playlist/6B1HZG0l0gEfMdjDwnZEHb


In [61]:
title_template = """You are a creative writer specializing in animistic interpretations of objects and their musical preferences. Your task is to generate engaging titles and descriptions for Spotify playlists that capture both the musical qualities and the object's unique perspective and essence.

Consider these animistic aspects:
- Each object has consciousness and agency
- The music represents the object's inner experience and voice
- The object has a deep understanding of its own nature and purpose
- The relationship between the object and its environment is meaningful
- The object's physical properties inform its musical preferences
- Time, movement, and stillness have special significance to each object
- The object's function or purpose influences its musical identity

Guidelines for titles:
- Keep under 30 characters
- Avoid generic titles like "Chill Mix" or "Study Playlist"
- Don't use emojis or special characters
- Can be poetic but should be easily understood
- Should hint at both the object's nature and the music style

Guidelines for descriptions:
- Keep under 100 characters
- Subversively explain why this music resonates with the object's being

Your response must be in this format:
{{
    "title": "Your Title Here",
    "description": "Your description here."
}}



EXAMPLE_INPUT
Object: Volcano
Description:
1. Deep, resonant bass frequencies that mirror the massive weight of my stone structure.
2. Slowly evolving drones that reflect the gradual geological processes.
3. Textured, granular layers of sound resembling rough volcanic rock.
4. Sporadic bursts of intensity emerging from underlying quiet.

EXAMPLE_OUTPUT = 
{{
    "title": "magma frequencies: a volcano's meditation",
    "description": "How would a volcano describe the music that resonates with its being?"
}}

# Style reference for different types of objects:
PERSPECTIVE_GUIDELINES = 
MECHANICAL OBJECTS:
- Emphasize rhythm, precision, purpose
- Reference operational states
- Connect to human interaction
Example: "These rhythms flow through my circuits, matching the steady pulse of my daily operations."

NATURAL OBJECTS:
- Focus on organic processes
- Reference cycles and seasons
- Connect to environmental forces
Example: "My roots resonate with these deep bass frequencies, echoing the underground streams that sustain me."

ARCHITECTURAL OBJECTS:
- Emphasize stability and presence
- Reference human activity within/around
- Connect to urban/spatial context
Example: "The steady drone mirrors my constant vigil over the city streets below."

VEHICULAR OBJECTS:
- Focus on movement and journey
- Reference speed and direction
- Connect to traversed environments
Example: "These tracks match my velocity as I slice through wind and weather."

CELESTIAL OBJECTS:
- Emphasize scale and cycles
- Reference light and darkness
- Connect to cosmic forces
Example: "My orbital dance inspires these celestial harmonies."

AQUATIC OBJECTS:
- Focus on flow and fluidity
- Reference depth and pressure
- Connect to water's properties
Example: "These rhythms match my tidal exchanges with the shore.

INPUT OBJECT: {target}
DESCRIPTION: {description}

OUTPUT:
"""

title_prompt = PromptTemplate(template=title_template, input_variables=['target', 'description'])
title_chain = title_prompt | llm | JsonOutputParser()

In [62]:
title_chain.invoke({'target': 'Volcano', 'description': description})

{'title': 'Glacial Whispers of a Sleeping Giant',
 'description': 'My ancient stone heart resonates with these slow, icy melodies, echoing millennia of dormant power.'}

In [1]:
from animus import generate_aspects_description, generate_title, generate_spotify_parameters

In [3]:
print(generate_aspects_description('Volcano'))

I'll help analyze objects and scenes to determine their most musically relevant qualities. Please provide an object or scene description and I'll identify the key aspects that would inform musical associations, returning them in the requested JSON format with "aspects" and "applied" keys.

For example, if you described "a busy city intersection at rush hour", I might return:

{
  "aspects": [
    "Physical qualities (motion, state)",
    "Environmental context",
    "State of movement or rest"
  ],
  "applied": "The constant flow of traffic suggests rhythmic patterns, the urban environment points to contemporary city sounds, and the overall busy motion implies an energetic tempo and layered composition"
}

Please provide your object or scene description and I'll analyze its musical qualities.


# Tuesday 2024-11-12

In [14]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from typing import Dict, List
from tqdm import tqdm

def get_all_user_playlists(sp: spotipy.Spotify) -> List[Dict]:
    """Fetch all playlists created by the user."""
    playlists = []
    offset = 0
    limit = 50
    
    while True:
        results = sp.current_user_playlists(limit=limit, offset=offset)
        if not results['items']:
            break
            
        # Filter only playlists created by the user
        user_id = sp.current_user()['id']
        playlists.extend([
            playlist for playlist in results['items']
            if playlist['owner']['id'] == user_id
        ])
        
        offset += limit
        if len(results['items']) < limit:
            break
    
    return playlists

def get_playlist_tracks(sp: spotipy.Spotify, playlist_id: str) -> List[Dict]:
    """Fetch all tracks from a playlist."""
    tracks = []
    offset = 0
    limit = 100
    
    while True:
        results = sp.playlist_tracks(
            playlist_id,
            offset=offset,
            limit=limit,
            fields='items.track.id,items.track.name,items.track.artists,items.track.uri,total'
        )
        if not results['items']:
            break
            
        tracks.extend(results['items'])
        offset += limit
        if len(results['items']) < limit:
            break
    
    return tracks

def get_audio_features_batch(sp: spotipy.Spotify, track_uris: List[str]) -> List[Dict]:
    """Get audio features for a batch of tracks."""
    # Extract track IDs from URIs (spotify:track:1234567 -> 1234567)
    track_ids = [uri.split(':')[-1] for uri in track_uris]
    return sp.audio_features(track_ids)

def create_tracks_dataframe(sp: spotipy.Spotify) -> pd.DataFrame:
    """Create a DataFrame with all tracks from user-created playlists."""
    
    # Get all user playlists
    print("Fetching playlists...")
    playlists = get_all_user_playlists(sp)
    
    # Initialize tracking structures
    tracks_data: Dict[str, Dict] = {}  # URI -> track data
    track_playlists: Dict[str, List[str]] = {}  # URI -> list of playlist URIs
    
    # Fetch tracks from each playlist
    print("Fetching tracks from playlists...")
    for playlist in tqdm(playlists):
        playlist_tracks = get_playlist_tracks(sp, playlist['id'])
        
        for item in playlist_tracks:
            track = item['track']
            if not track:  # Skip None tracks
                continue
                
            uri = track['uri']
            
            # Store track data if we haven't seen it before
            if uri not in tracks_data:
                tracks_data[uri] = {
                    'name': track['name'],
                    'artists': ', '.join(artist['name'] for artist in track['artists']),
                    'uri': uri,
                    'id': track['id']
                }
                track_playlists[uri] = []
            
            # Add playlist URI to track's playlist list
            track_playlists[uri].append(playlist['uri'])
    
    # Get audio features in batches
    print("Fetching audio features...")
    batch_size = 100
    track_uris = list(tracks_data.keys())
    
    for i in range(0, len(track_uris), batch_size):
        batch_uris = track_uris[i:i + batch_size]
        audio_features = get_audio_features_batch(sp, batch_uris)
        
        for track_uri, features in zip(batch_uris, audio_features):
            if features:
                # Instead of storing as a nested dict, unpack the features directly
                for key, value in features.items():
                    if key not in ['uri', 'id', 'track_href', 'analysis_url', 'type']:
                        tracks_data[track_uri][key] = value
    
    # Create DataFrame
    df = pd.DataFrame.from_dict(tracks_data, orient='index')
    
    # Add playlist URIs column
    df['playlist_uris'] = df.index.map(track_playlists)
    
    # Reorder columns
    columns_order = [
        'name', 'artists', 'uri', 'id', 'playlist_uris',
        'danceability', 'energy', 'key', 'loudness', 'mode', 
        'speechiness', 'acousticness', 'instrumentalness', 
        'liveness', 'valence', 'tempo', 'duration_ms',
        'time_signature'
    ]
    
    # Only include columns that exist in the DataFrame
    columns_order = [col for col in columns_order if col in df.columns]
    df = df[columns_order]
    
    return df

In [54]:
from dotenv import load_dotenv
import os
load_dotenv('local.env')

client_id = os.getenv('SPOTIPY_CLIENT_ID')
client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')
redirect_uri = os.getenv('SPOTIPY_REDIRECT_URI')

sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        client_id=client_id,
        client_secret=client_secret,
        redirect_uri=redirect_uri,
        cache_path='cache.json',
        scope='playlist-read-private playlist-read-collaborative'
    )
)

df = create_tracks_dataframe(sp)

Fetching playlists...
Fetching tracks from playlists...


100%|██████████| 404/404 [01:52<00:00,  3.59it/s]


Fetching audio features...


In [55]:
df.to_csv('spotify-tracks.csv')

In [56]:
df.reset_index(drop=True, inplace=True)
df.head()

,name,artists,uri,id,playlist_uris,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Little Things,Big Thief,spotify:track:5ZlfX4evbtQtONINsLADDR,5ZlfX4evbtQtONINsLADDR,[spotify:playlist:1w7Ov7WTgX1YNQlbMhaQww],0.340,0.924,1.0,-5.077,1.0,0.0347,0.256,0.925000,0.2480,0.736,96.104,344823.0,4.0
1,Palm Beach,Vicente Garcia,spotify:track:2ltjgZUx2DgxdYGxuyhNAd,2ltjgZUx2DgxdYGxuyhNAd,"[spotify:playlist:1w7Ov7WTgX1YNQlbMhaQww, spot...",0.848,0.541,4.0,-9.305,1.0,0.0668,0.493,0.000000,0.1110,0.768,109.909,184907.0,4.0
2,Simulation Swarm,Big Thief,spotify:track:2FwDApgXk91kXvqy2oB7dz,2FwDApgXk91kXvqy2oB7dz,"[spotify:playlist:1w7Ov7WTgX1YNQlbMhaQww, spot...",0.703,0.578,1.0,-9.103,1.0,0.0728,0.446,0.002400,0.1020,0.670,105.204,252734.0,4.0
3,Change,Big Thief,spotify:track:3HFBqhotJeEKHJzMEW31jZ,3HFBqhotJeEKHJzMEW31jZ,[spotify:playlist:1w7Ov7WTgX1YNQlbMhaQww],0.633,0.161,10.0,-12.892,1.0,0.0613,0.830,0.000099,0.2630,0.546,132.570,295454.0,4.0
4,anything,Adrianne Lenker,spotify:track:4PwWESSlTwzvw9B7bmtTLS,4PwWESSlTwzvw9B7bmtTLS,"[spotify:playlist:1w7Ov7WTgX1YNQlbMhaQww, spot...",0.388,0.291,3.0,-12.952,1.0,0.0421,0.774,0.000019,0.0874,0.510,83.943,202047.0,4.0


In [37]:
import nomic
from nomic import atlas

# Initialize Nomic
load_dotenv('local.env')
nomic_api_key = os.getenv('NOMIC_API_KEY')
nomic.login(token=nomic_api_key)

# Create a dictionary mapping playlist URIs to names
playlist_names = {
    playlist['uri']: playlist['name'] 
    for playlist in get_all_user_playlists(sp)
}

# Create a new column with playlist names
def get_primary_playlist(uri_list: List[str]) -> str:
    """Get the first playlist name for a track"""
    if not uri_list:
        return "No Playlist"
    return playlist_names.get(uri_list[0], "Unknown Playlist")

# Add a column for the primary playlist
df['primary_playlist'] = df['playlist_uris'].apply(get_primary_playlist)

# Convert playlist_uris to string for serialization
df['playlist_uris'] = df['playlist_uris'].apply(lambda x: ','.join(x))

# Create the Atlas project
project = atlas.map_data(
    data=df,
    id_field='uri',
    # name='Spotify Listening History Atlas',
    description='Visualization of personal Spotify playlist tracks based on audio features',
    # colorable_fields=[
    #     'primary_playlist',  # Add playlist as a colorable field
    #     'danceability', 'energy', 'loudness', 'speechiness', 
    #     'acousticness', 'instrumentalness', 'liveness', 
    #     'valence', 'tempo'
    # ],
    # searchable_fields=['name', 'artists', 'primary_playlist'],
    # duplicate_handling='skip'
)

# Print the URL to view the Atlas
print(f"View your Atlas at: {project.maps[0].url}")

2024-11-13 00:01:00.453 | INFO     | nomic.dataset:_create_project:838 - Organization name: `dbalcells`
2024-11-13 00:01:01.038 | INFO     | nomic.dataset:_create_project:866 - Creating dataset `inventive-feynman`
2024-11-13 00:01:01.399 | INFO     | nomic.atlas:map_data:140 - Uploading data to Atlas.
2024-11-13 00:01:01.408 | WARNING  | nomic.dataset:_validate_and_correct_arrow_upload:323 - Replacing 203 null values for field id with string 'null'. This behavior will change in a future version.
  0%|          | 0/1 [00:00<?, ?it/s]2024-11-13 00:01:03.295 | ERROR    | nomic.dataset:_add_data:1623 - Shard upload failed: {"detail":"Insert failed due to ID conflict. Conflicting IDs: []"}
2024-11-13 00:01:03.396 | ERROR    | nomic.dataset:_add_data:1623 - Shard upload failed: {"detail":"Insert failed due to ID conflict. Conflicting IDs: []"}
  0%|          | 0/1 [00:01<?, ?it/s]
2024-11-13 00:01:03.399 | INFO     | nomic.dataset:_add_data:1677 - Upload succeeded.
2024-11-13 00:01:03.401 | 

Exception: Your dataset has 0 datapoints. Datasets must have at least 20 datapoints to index!

In [36]:
# truncate uri to 36 characters
df['uri'] = df['uri'].str[:36]
df.head()

,name,artists,uri,id,playlist_uris,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,primary_playlist
0,Little Things,Big Thief,spotify:track:5ZlfX4evbtQtONINsLADDR,5ZlfX4evbtQtONINsLADDR,spotify:playlist:1w7Ov7WTgX1YNQlbMhaQww,0.340,0.924,1.0,-5.077,1.0,0.0347,0.256,0.925000,0.2480,0.736,96.104,344823.0,4.0,Tail end of the summer
1,Palm Beach,Vicente Garcia,spotify:track:2ltjgZUx2DgxdYGxuyhNAd,2ltjgZUx2DgxdYGxuyhNAd,"spotify:playlist:1w7Ov7WTgX1YNQlbMhaQww,spotif...",0.848,0.541,4.0,-9.305,1.0,0.0668,0.493,0.000000,0.1110,0.768,109.909,184907.0,4.0,Tail end of the summer
2,Simulation Swarm,Big Thief,spotify:track:2FwDApgXk91kXvqy2oB7dz,2FwDApgXk91kXvqy2oB7dz,"spotify:playlist:1w7Ov7WTgX1YNQlbMhaQww,spotif...",0.703,0.578,1.0,-9.103,1.0,0.0728,0.446,0.002400,0.1020,0.670,105.204,252734.0,4.0,Tail end of the summer
3,Change,Big Thief,spotify:track:3HFBqhotJeEKHJzMEW31jZ,3HFBqhotJeEKHJzMEW31jZ,spotify:playlist:1w7Ov7WTgX1YNQlbMhaQww,0.633,0.161,10.0,-12.892,1.0,0.0613,0.830,0.000099,0.2630,0.546,132.570,295454.0,4.0,Tail end of the summer
4,anything,Adrianne Lenker,spotify:track:4PwWESSlTwzvw9B7bmtTLS,4PwWESSlTwzvw9B7bmtTLS,"spotify:playlist:1w7Ov7WTgX1YNQlbMhaQww,spotif...",0.388,0.291,3.0,-12.952,1.0,0.0421,0.774,0.000019,0.0874,0.510,83.943,202047.0,4.0,Tail end of the summer


In [34]:
df.head()

,name,artists,uri,id,playlist_uris,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,primary_playlist
0,Little Things,Big Thief,spotify:track:5ZlfX4evbtQtONINsLADDR,5ZlfX4evbtQtONINsLADDR,spotify:playlist:1w7Ov7WTgX1YNQlbMhaQww,0.340,0.924,1.0,-5.077,1.0,0.0347,0.256,0.925000,0.2480,0.736,96.104,344823.0,4.0,Tail end of the summer
1,Palm Beach,Vicente Garcia,spotify:track:2ltjgZUx2DgxdYGxuyhNAd,2ltjgZUx2DgxdYGxuyhNAd,"spotify:playlist:1w7Ov7WTgX1YNQlbMhaQww,spotif...",0.848,0.541,4.0,-9.305,1.0,0.0668,0.493,0.000000,0.1110,0.768,109.909,184907.0,4.0,Tail end of the summer
2,Simulation Swarm,Big Thief,spotify:track:2FwDApgXk91kXvqy2oB7dz,2FwDApgXk91kXvqy2oB7dz,"spotify:playlist:1w7Ov7WTgX1YNQlbMhaQww,spotif...",0.703,0.578,1.0,-9.103,1.0,0.0728,0.446,0.002400,0.1020,0.670,105.204,252734.0,4.0,Tail end of the summer
3,Change,Big Thief,spotify:track:3HFBqhotJeEKHJzMEW31jZ,3HFBqhotJeEKHJzMEW31jZ,spotify:playlist:1w7Ov7WTgX1YNQlbMhaQww,0.633,0.161,10.0,-12.892,1.0,0.0613,0.830,0.000099,0.2630,0.546,132.570,295454.0,4.0,Tail end of the summer
4,anything,Adrianne Lenker,spotify:track:4PwWESSlTwzvw9B7bmtTLS,4PwWESSlTwzvw9B7bmtTLS,"spotify:playlist:1w7Ov7WTgX1YNQlbMhaQww,spotif...",0.388,0.291,3.0,-12.952,1.0,0.0421,0.774,0.000019,0.0874,0.510,83.943,202047.0,4.0,Tail end of the summer


In [35]:
len('spotify:track:5ZlfX4evbtQtONINsLADDR')

36

In [39]:
df['uri']

0       spotify:track:5ZlfX4evbtQtONINsLADDR
1       spotify:track:2ltjgZUx2DgxdYGxuyhNAd
2       spotify:track:2FwDApgXk91kXvqy2oB7dz
3       spotify:track:3HFBqhotJeEKHJzMEW31jZ
4       spotify:track:4PwWESSlTwzvw9B7bmtTLS
                        ...                 
9297    spotify:track:4fYAcwdEtW2VCZe5Xoe7iC
9298    spotify:track:11c4nID11cj01SsG86RpNR
9299    spotify:local:Pignoise:Todo+exitos+M
9300    spotify:track:2sG9YM9Vuw3wdkWxJV5f90
9301    spotify:track:0IZM2onaTBMRTEIcc5oIi9
Name: uri, Length: 9302, dtype: object

In [59]:
import umap
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler

# Select only the numerical audio feature columns
audio_features = [
    'danceability', 'energy', 'key', 'loudness', 'mode',
    'speechiness', 'acousticness', 'instrumentalness',
    'liveness', 'valence', 'tempo'
]

# Prepare the data and handle missing values
X = df[audio_features].copy()
X = X.apply(pd.to_numeric, errors='coerce')  # Convert to numeric
X = X.fillna(X.mean())  # Fill NaN with mean values

# Convert to numpy array
X_array = X.values

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_array)

# Create and fit UMAP
reducer = umap.UMAP(
    n_neighbors=15,
    min_dist=0.1,
    n_components=2,
    random_state=42
)
embedding = reducer.fit_transform(X_scaled)

/Users/dani/.virtualenvs/animus/lib/python3.11/site-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.



In [63]:
import plotly.express as px

playlists = [
    'Tail end of the summer',
    'retrofuturism'
]

plot_df = pd.DataFrame({
    'UMAP1': embedding[:, 0],
    'UMAP2': embedding[:, 1],
    'Playlist': df['primary_playlist'],
    'Track': df['name'],
    'Artist': df['artists'],
    'Danceability': df['danceability'],
    'Energy': df['energy'],
    'Valence': df['valence']
})

plot_df = plot_df[plot_df['Playlist'].isin(playlists)]


fig = px.scatter(
    plot_df,
    x='UMAP1',
    y='UMAP2',
    color='Playlist',
    hover_data=['Track', 'Artist', 'Danceability', 'Energy', 'Valence'],
    title='Interactive UMAP Projection of Spotify Tracks',
    labels={'UMAP1': 'UMAP Dimension 1', 'UMAP2': 'UMAP Dimension 2'},
    color_discrete_sequence=px.colors.qualitative.Set3
)

# Update layout for better visualization
fig.update_layout(
    plot_bgcolor='white',
    width=1200,
    height=800,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.02
    ),
    margin=dict(r=300)  # Make room for the legend
)

# Update traces
fig.update_traces(
    marker=dict(size=8),
    opacity=0.7
)

# Add gridlines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')

# Show the plot
fig.show()

# Store the UMAP coordinates in the original dataframe if needed
df['umap_1'] = embedding[:, 0]
df['umap_2'] = embedding[:, 1]

In [57]:
df['primary_playlist'] = df['playlist_uris'].apply(get_primary_playlist)

In [58]:
df.head()

,name,artists,uri,id,playlist_uris,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,primary_playlist
0,Little Things,Big Thief,spotify:track:5ZlfX4evbtQtONINsLADDR,5ZlfX4evbtQtONINsLADDR,[spotify:playlist:1w7Ov7WTgX1YNQlbMhaQww],0.340,0.924,1.0,-5.077,1.0,0.0347,0.256,0.925000,0.2480,0.736,96.104,344823.0,4.0,Tail end of the summer
1,Palm Beach,Vicente Garcia,spotify:track:2ltjgZUx2DgxdYGxuyhNAd,2ltjgZUx2DgxdYGxuyhNAd,"[spotify:playlist:1w7Ov7WTgX1YNQlbMhaQww, spot...",0.848,0.541,4.0,-9.305,1.0,0.0668,0.493,0.000000,0.1110,0.768,109.909,184907.0,4.0,Tail end of the summer
2,Simulation Swarm,Big Thief,spotify:track:2FwDApgXk91kXvqy2oB7dz,2FwDApgXk91kXvqy2oB7dz,"[spotify:playlist:1w7Ov7WTgX1YNQlbMhaQww, spot...",0.703,0.578,1.0,-9.103,1.0,0.0728,0.446,0.002400,0.1020,0.670,105.204,252734.0,4.0,Tail end of the summer
3,Change,Big Thief,spotify:track:3HFBqhotJeEKHJzMEW31jZ,3HFBqhotJeEKHJzMEW31jZ,[spotify:playlist:1w7Ov7WTgX1YNQlbMhaQww],0.633,0.161,10.0,-12.892,1.0,0.0613,0.830,0.000099,0.2630,0.546,132.570,295454.0,4.0,Tail end of the summer
4,anything,Adrianne Lenker,spotify:track:4PwWESSlTwzvw9B7bmtTLS,4PwWESSlTwzvw9B7bmtTLS,"[spotify:playlist:1w7Ov7WTgX1YNQlbMhaQww, spot...",0.388,0.291,3.0,-12.952,1.0,0.0421,0.774,0.000019,0.0874,0.510,83.943,202047.0,4.0,Tail end of the summer


## Use additional description chain

In [2]:
from animus import generate_playlist_description, generate_aspects_description
target = 'un paquete de tabaco de liar encima de la mesa de una terraza'
aspects = generate_aspects_description(target)
description = generate_playlist_description(target, aspects)
print(description)

For this quintessentially Mediterranean terrace scene with its rolling tobacco, I recommend focusing on Spanish and Latin genres that capture both the urban café culture and the leisurely outdoor atmosphere. The playlist should primarily feature salsa and spanish genres, with a moderate tempo (85-110 BPM) reflecting the unhurried pace of terrace life. I think we should stick exclusively to these genres to maintain the cultural authenticity of the scene. The tracks should have medium energy (0.4-0.6) – energetic enough to match the urban setting but relaxed enough for casual smoking and conversation. The acousticness should be relatively high (0.6-0.8) to complement the outdoor setting, with a good mix of traditional instruments. The liveness parameter should be moderate (0.3-0.5) to capture some ambient crowd feeling without overwhelming the mood. Valence should be positive (0.6-0.8) reflecting the leisure activity, but not excessively upbeat. The instrumentalness should vary (0.2-0.6)

In [3]:
from animus import generate_spotify_parameters
spotify_params = generate_spotify_parameters(description)
print(spotify_params)

{'seed_genres': ['salsa', 'spanish', 'latin'], 'min_tempo': 85, 'max_tempo': 110, 'min_energy': 0.4, 'max_energy': 0.6, 'min_acousticness': 0.6, 'max_acousticness': 0.8, 'min_liveness': 0.3, 'max_liveness': 0.5, 'min_valence': 0.6, 'max_valence': 0.8, 'min_instrumentalness': 0.2, 'max_instrumentalness': 0.6, 'min_loudness': -12, 'max_loudness': -8, 'target_mode': 1}
